# Install and import libraries

In [1]:
# Python version 3.9
# Java 8
# Spark 3.2.0

# run below code if it's your first time
# !pip install pyspark

# If you are using a windows system, you may experience the below error, so hadoop 3.2.2 should be installed
# Follow the instructions writen by Jialing

In [2]:
# run below code if it's your first time

# !pip install findspark

In [3]:
import glob
import pandas as pd
import numpy as np
from pymsfilereader import MSFileReader

### Define data path

In [4]:
# TODO: change to \\ for Windows specific
#path = ".\\data"
path = r"F:\0_RAW_to_CSV\LiverCancerReproj\LC_C18\HKULiver_Post29wk_HCCAMCLiver_LCMS_MiceNL_S08"

### Get list of groups

In [5]:
# TODO: change to \\ for Windows specific
groups = glob.glob(f"{path}\\*")

In [6]:
def process_one_file(in_file):
    rawfile1=MSFileReader(in_file)
    frame = []
    try:
        rawfile=MSFileReader(in_file)
        y = int(in_file.split("\\")[-1].split('.')[0].split('_')[-1])
        file_name = "_".join(in_file.split("\\")[-1].split('.')[0].split('_')[:-1])
    
        for i in range(rawfile.FirstSpectrumNumber, rawfile.LastSpectrumNumber + 1):
            labels, flags = rawfile.GetLabelData(i)
            
            cur_df = pd.DataFrame({'filename': file_name, 
                                   'x': i, 
                                   'y': y,
                                   'mass': labels.mass, 
                                   'intensity': labels.intensity, 
                                   'noise': labels.noise, 
                                   'resolution': labels.resolution})
            
            frame.append(cur_df)
        
        return frame
    except IOError:
        print("file error {in_file}".format(in_file=in_file))

In [7]:
count = 0
for file in groups:
    print("processing file: ", file)
    to_csv = process_one_file(in_file=file)
    to_csv1=to_csv[0]
    for i in range(1,len(to_csv)):
        cat = to_csv[i] #.drop(columns=["y","filename", "resolution"])
        to_csv1 = pd.concat([to_csv1, cat])
    if count == 0:
        big_cat = to_csv1
        count += 1
    elif count > 0:
        big_cat = pd.concat([big_cat, to_csv1])

processing file:  F:\0_RAW_to_CSV\LiverCancerReproj\LC_C18\HKULiver_Post29wk_HCCAMCLiver_LCMS_MiceNL_S08\HKULiver_Post29wk_HCCAMCLiver_LCMS_MiceNL_S08_001.raw


In [8]:
big_cat.to_csv(r'F:\0_RAW_to_CSV\LiverCancerReproj\LC_C18\HKULiver_Post29wk_HCCAMCLiver_LCMS_MiceNL_S08\HKULiver_Post29wk_HCCAMCLiver_LCMS_MiceNL_S08.csv')